The purpose of this notebook is to extract and inspect the precip estimates based on historical and future WRF AMS for a specific location. 

Test location:
61.14N, 152.62W

In [42]:
# setup
import os, time
import numpy as np
import pandas as pd
import rasterio as rio
import xarray as xr
from pyproj import Transformer

# directories
data_dir = "/workspace/Shared/Tech_Projects/DOT/project_data"
wrf_dir = os.path.join(data_dir, "wrf_pcpt")

# WGS84 coordinates from example of invalid bounds
wgs84_coords = (-152.62, 61.14)
# WRF CRS
wrf_crs = '+units=m +proj=stere +lat_ts=64.0 +lon_0=-152.0 +lat_0=90.0 +x_0=0 +y_0=0 +a=6370000 +b=6370000'
transformer = Transformer.from_proj("EPSG:4326", wrf_crs, always_xy=True)
wrf_coords = transformer.transform(*wgs84_coords)

In [38]:
oid_fp = os.path.join(wrf_dir, "output_interval_durations", "pcpt_{}_{}_sum_wrf_{}_{}_intervals.nc")
def get_pf(fp, xc=wrf_coords[0], yc=wrf_coords[1]):
    pf_ds = xr.open_dataset(fp)
    return np.round(pf_ds.sel(xc=xc, yc=yc, method="nearest").pf.values, 3)

durations = ['2h','3h','6h','12h','24h','2d','3d','4d','7d','10d','20d','30d','45d','60d',]
hist = {duration: get_pf(oid_fp.format("GFDL-CM3", "historical", duration, "1979-2015")) for duration in durations}
rcp85 = {duration: get_pf(oid_fp.format("GFDL-CM3", "rcp85", duration, "2020-2049")) for duration in durations}
# make tables
ris = np.array([2,5,10,25,50,100,200,500,1000]).astype("str") 
colnames = [ri + "yr" for ri in ris]
hist_df = pd.DataFrame.from_dict(hist, orient="index", columns = colnames)
rcp85_df = pd.DataFrame.from_dict(rcp85, orient="index", columns = colnames)

In [90]:
!ls /workspace/Shared/Tech_Projects/DOT/project_data/wrf_pcpt/combined-fixedci

pcpt_GFDL-CM3_sum_wrf_10d_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_10d_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_10d_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_12h_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_12h_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_12h_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_20d_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_20d_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_20d_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_24h_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_24h_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_24h_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_2h_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_2h_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_2h_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_30d_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_30d_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_30d_2080-2099_combined.nc
pcpt_GFDL-CM3_sum_wrf_3d_2020-2049_combined.nc
pcpt_GFDL-CM3_sum_wrf_3d_2050-2079_combined.nc
pcpt_GFDL-CM3_sum_wrf_3d_2080-2099_combined.n

In [87]:
new_wgs84 = (-153.75, 62.24)
new_wrf_coords = transformer.transform(*new_wgs84)
oi_ci_fp = os.path.join(wrf_dir, "interval_diffs-fixedci", "pcpt_{}_{}_sum_wrf_{}_{}_intervals.nc")
oi_ci_ds = xr.open_dataset(oi_ci_fp.format("GFDL-CM3", "rcp85", "6h", "2020-2049"))
oi_ci_sel = oi_ci_ds.sel(xc=new_wrf_coords[0], yc=new_wrf_coords[1], method="nearest")
print(np.round(oi_ci_sel["pf-upper"].values, 3))
print(np.round(oi_ci_sel["pf"].values, 3))
print(np.round(oi_ci_sel["pf-lower"].values, 3))

[0.059 0.098 0.156 0.308 0.518 0.835 1.342 2.395 3.584]
[0.498 0.642 0.757 0.926 1.074 1.241 1.432 1.726 1.986]
[-0.047 -0.084 -0.129 -0.224 -0.323 -0.449 -0.604 -0.859 -1.091]


In [89]:
a14_transformer = Transformer.from_crs(4326, 3338, always_xy=True)
new_a14_coords = a14_transformer.transform(*new_wgs84)
oi_ci_fp = os.path.join(wrf_dir, "warped-fixedci", "pcpt_{}_sum_wrf_{}_{}_warped.nc")
oi_ci_ds = xr.open_dataset(oi_ci_fp.format("GFDL-CM3", "6h", "2020-2049"))
oi_ci_sel = oi_ci_ds.sel(xc=new_a14_coords[0], yc=new_a14_coords[1], method="nearest")
print(np.round(oi_ci_sel["pf-upper"].values, 3))
print(np.round(oi_ci_sel["pf"].values, 3))
print(np.round(oi_ci_sel["pf-lower"].values, 3))

[ 0.09   0.172  0.295  0.622  1.131  1.994  3.616  7.535 12.977]
[1.202 1.283 1.431 1.73  2.048 2.467 3.015 4.004 5.017]
[-0.064 -0.135 -0.229 -0.435 -0.678 -1.018 -1.483 -2.341 -3.241]


In [94]:
comb_ci_fp = os.path.join(wrf_dir, "combined-fixedci", "pcpt_{}_sum_wrf_{}_{}_combined.nc")
comb_ci_ds = xr.open_dataset(comb_ci_fp.format("GFDL-CM3", "6h", "2020-2049"))
comb_ci_sel = comb_ci_ds.sel(xc=new_a14_coords[0], yc=new_a14_coords[1], method="nearest")
print(np.round(comb_ci_sel["pf-upper"].values, 3))
print(np.round(comb_ci_sel["pf"].values, 3))
print(np.round(comb_ci_sel["pf-lower"].values, 3))

[ 1056.65   1566.602  2100.047  3102.137  4183.254  5661.226  7909.92
 12272.133 17087.719]
[ 1053.12   1559.837  2088.433  3077.639  4138.734  5582.726  7767.555
 11975.491 16576.795]
[ 1050.582  1554.532  2079.417  3060.52   4112.052  5542.651  7709.181
 11883.32  16449.203]


<xarray.Dataset>
Dimensions:   (interval: 9, xc: 262, yc: 262)
Coordinates:
  * xc        (xc) float64 -2.61e+06 -2.59e+06 -2.57e+06 ... 2.59e+06 2.61e+06
  * yc        (yc) float64 -1.824e+05 -2.024e+05 ... -5.382e+06 -5.402e+06
  * interval  (interval) float64 2.0 5.0 10.0 25.0 50.0 100.0 200.0 500.0 1e+03
Data variables:
    pf        (interval, xc, yc) float32 ...
    pf-upper  (interval, xc, yc) float32 ...
    pf-lower  (interval, xc, yc) float32 ...

In [33]:
hist_df

,2yr,5yr,10yr,25yr,50yr,100yr,200yr,500yr,1000yr
2h,0.206,0.269,0.308,0.353,0.385,0.414,0.441,0.475,0.499
3h,0.316,0.412,0.472,0.544,0.594,0.642,0.688,0.745,0.786
6h,0.418,0.537,0.604,0.679,0.727,0.770,0.808,0.851,0.881
12h,0.626,0.796,0.875,0.948,0.987,1.017,1.040,1.062,1.075
24h,0.810,1.093,1.256,1.438,1.557,1.663,1.759,1.871,1.946
2d,1.085,1.451,1.634,1.814,1.919,2.003,2.072,2.143,2.185
3d,1.203,1.597,1.859,2.193,2.442,2.690,2.940,3.270,3.522
4d,1.341,1.734,2.011,2.382,2.673,2.976,3.292,3.732,4.083
7d,1.652,2.039,2.264,2.518,2.686,2.838,2.975,3.138,3.248
10d,2.147,2.612,2.870,3.147,3.324,3.477,3.611,3.764,3.862


In [34]:
rcp85_df

,2yr,5yr,10yr,25yr,50yr,100yr,200yr,500yr,1000yr
2h,0.261,0.324,0.365,0.415,0.452,0.488,0.524,0.570,0.604000
3h,0.386,0.492,0.571,0.683,0.776,0.878,0.990,1.154,1.292000
6h,0.516,0.663,0.768,0.911,1.026,1.146,1.274,1.455,1.602000
12h,0.741,0.963,1.103,1.273,1.393,1.509,1.620,1.760,1.862000
24h,1.035,1.318,1.459,1.598,1.679,1.745,1.798,1.854,1.887000
2d,1.297,1.656,1.847,2.043,2.163,2.264,2.349,2.442,2.500000
3d,1.529,2.109,2.521,3.074,3.510,3.966,4.444,5.113,5.649000
4d,1.639,2.233,2.653,3.213,3.653,4.110,4.588,5.252,5.783000
7d,2.094,2.706,3.109,3.616,3.991,4.361,4.728,5.210,5.573000
10d,2.657,3.627,4.317,5.244,5.976,6.742,7.546,8.672,9.576000


In [36]:
del_fp = os.path.join(wrf_dir, "deltas", "pcpt_{}_sum_wrf_{}_{}_deltas.nc")
def get_pf(fp, xc=wrf_coords[0], yc=wrf_coords[1]):
    pf_ds = xr.open_dataset(fp)
    return np.round(pf_ds.sel(xc=xc, yc=yc, method="nearest").pf.values, 3)

durations = ['2h','3h','6h','12h','24h','2d','3d','4d','7d','10d','20d','30d','45d','60d',]
deltas = {duration: get_pf(del_fp.format("GFDL-CM3", duration, "2020-2049")) for duration in durations}
# make tables
del_df = pd.DataFrame.from_dict(deltas, orient="index", columns = colnames)

In [37]:
del_df

,2yr,5yr,10yr,25yr,50yr,100yr,200yr,500yr,1000yr
2h,1.269,1.203,1.184,1.176,1.176,1.180,1.187,1.199,1.210
3h,1.223,1.193,1.209,1.256,1.306,1.368,1.439,1.550,1.645
6h,1.237,1.235,1.271,1.343,1.411,1.489,1.577,1.709,1.819
12h,1.184,1.209,1.261,1.343,1.412,1.483,1.557,1.657,1.732
24h,1.279,1.206,1.162,1.112,1.079,1.049,1.022,0.991,0.970
2d,1.195,1.141,1.130,1.126,1.127,1.130,1.134,1.140,1.144
3d,1.270,1.321,1.356,1.402,1.438,1.474,1.512,1.563,1.604
4d,1.222,1.288,1.319,1.349,1.366,1.381,1.394,1.407,1.416
7d,1.268,1.327,1.373,1.436,1.486,1.537,1.589,1.660,1.716
10d,1.238,1.389,1.504,1.666,1.798,1.939,2.089,2.304,2.479


This begs the question of why there are cases of final estimates decreasing with increasing return intervals. Since the Atlas 14 data are monotonic in this regard, the deltas need to be non-monotonic to result in non-monotonic series within a return interval. This suggests that the issue could be in the warped deltas. Check those:

In [69]:
# transform to 3338 coords
transformer = Transformer.from_crs(4326, 3338, always_xy=True)
epsg3338_coords = transformer.transform(*wgs84_coords)

warp_fp = os.path.join(wrf_dir, "warped", "pcpt_{}_sum_wrf_{}_{}_warped.nc")
warped = {
    duration: get_pf(
        warp_fp.format("GFDL-CM3", duration, "2020-2049"),
        xc=epsg3338_coords[0],
        yc=epsg3338_coords[1],
    )
    for duration in durations
}
# make tables
warp_df = pd.DataFrame.from_dict(warped, orient="index", columns = colnames)

In [70]:
warp_df

,2yr,5yr,10yr,25yr,50yr,100yr,200yr,500yr,1000yr
2h,1.040,1.024,1.054,1.122,1.189,1.270,1.366,1.517,1.652
3h,1.078,1.043,1.054,1.086,1.118,1.154,1.194,1.250,1.294
6h,1.080,1.061,1.085,1.138,1.190,1.251,1.320,1.424,1.513
12h,1.103,1.057,1.052,1.063,1.078,1.099,1.123,1.160,1.191
24h,1.109,1.122,1.154,1.208,1.255,1.305,1.358,1.431,1.489
2d,1.161,1.102,1.093,1.104,1.124,1.153,1.190,1.251,1.307
3d,1.112,1.080,1.056,1.025,1.005,0.989,0.976,0.966,0.964
4d,1.182,1.173,1.143,1.093,1.051,1.008,0.965,0.909,0.869
7d,1.232,1.182,1.091,0.946,0.831,0.720,0.617,0.496,0.417
10d,1.276,1.286,1.221,1.085,0.963,0.838,0.717,0.570,0.473


In [ ]:
transformer = Transformer.from_proj("EPSG:4326", wrf_crs, always_xy=True)
wrf_coords = transformer.transform(*wgs84_coords)

In [56]:
# query original Atlas 14 values

# get row/column of WGS84 example coordinates
# open sample file
a14_pf = rio.open(os.path.join(data_dir, "NOAA_Atlas14", "raw", "warped", "ak1000yr04da_ams.tif"))
# transform to 3338
transformer = Transformer.from_crs(4326, 3338, always_xy=True)
a14_coords = transformer.transform(*wgs84_coords)
a14_rc = a14_pf.index(*a14_coords)
window = ((a14_rc[0], a14_rc[0] + 1), (a14_rc[1], a14_rc[1] + 1))


def get_a14(fp, window=window):
    a14_pf = rio.open(fp)
    pf = a14_pf.read(1, window=window) / 1000
    return round(pf[0][0], 3)


a14_durations = ["02ha", "03ha", "06ha", "12ha", "24ha", "02da", "03da", "04da", "07da", "10da", "20da", "30da", "45da", "60da"]
a14_fp = os.path.join(data_dir, "NOAA_Atlas14", "raw", "warped", "ak{}yr{}_ams.tif") 

out = [[get_a14(a14_fp.format(ri, dur)) for ri in ris] for dur in a14_durations]


RasterioIOError: /workspace/Shared/Tech_Projects/DOT/project_data/NOAA_Atlas14/raw/warped/ak2yr02da_ams.tif: No such file or directory

NameError: name 'a14_fp' is not defined

In [62]:
!ls /workspace/Shared/Tech_Projects/DOT/project_data/wrf_pcpt/warped

pcpt_GFDL-CM3_sum_wrf_10d_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_10d_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_10d_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_12h_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_12h_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_12h_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_20d_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_20d_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_20d_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_24h_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_24h_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_24h_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_2d_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_2d_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_2d_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_2h_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_2h_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_2h_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_30d_2020-2049_warped.nc
pcpt_GFDL-CM3_sum_wrf_30d_2050-2079_warped.nc
pcpt_GFDL-CM3_sum_wrf_30d_2080-2099_warped.nc
pcpt_GFDL-CM3_sum_wrf_3d_2020-2049_warpe

In [54]:
# del_fp = os.path.join(wrf_dir, "deltas", "pcpt_{}_sum_wrf_{}_{}_deltas.nc")
# # can use same function for getting durations
# d4_del_ds = xr.open_dataset(del_fp.format("GFDL-CM3", "4d", "2020-2049"))
# d4_deltas = d4_del_ds.sel(xc=wrf_coords[0], yc=wrf_coords[1], method="nearest").pf.values
data_dir

'/workspace/Shared/Tech_Projects/DOT/project_data'

In [59]:
!ls /workspace/Shared/Tech_Projects/DOT/project_data/

ancillary	     fai_acis_prcp_obs.csv  shapefiles	 validate_wrf_estimates
compare_atlas14_wrf  NOAA_Atlas14	    testing_eva  wrf_pcpt
